# Exercise 2.2

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import scipy

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Load dataset

In [75]:
trainset = torchvision.datasets.MNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])

train_loader = torch.utils.data.DataLoader(trainset,
                             batch_size=4, shuffle=True)

val_loader = torch.utils.data.DataLoader(validationset,
                             batch_size=1, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
                             batch_size=1, shuffle=True)

### Model

In [76]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 4)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 4)
        self.fc1 = nn.Linear(16 * 4 * 4, 84)
        #self.fc2 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.leaky_relu(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
        x = self.fc2(x)

        return x

model = Net();

# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0002)

writer = SummaryWriter()

### Training

In [77]:
best_accuracy = 0
best_net = 0

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train", loss, epoch)
        loss.backward()
        optimizer.step()
        if(i % 100 == 99):
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

    correct = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data
        labels = labels

        outputs = model(inputs)
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        best_net = copy.deepcopy(model)
        print(" (new best)")

print('Finished Training')
writer.flush()

Epoch 1 [12000/12000] - Loss: 0.11779198795557022365, accuracy:  0.9645
 (new best)
Epoch 2 [12000/12000] - Loss: 0.03410159051418304426, accuracy:  0.9778333333333333
 (new best)
Epoch 3 [12000/12000] - Loss: 0.0397364161908626569, accuracy:  0.9798333333333333
 (new best)
Epoch 4 [12000/12000] - Loss: 0.00022887454542797064, accuracy:  0.9855
 (new best)
Epoch 5 [12000/12000] - Loss: 0.02626394480466842764, accuracy:  0.9853333333333333
Finished Training


### Test

In [78]:
correct = 0
total = 0
for i, data in enumerate(test_loader, 0):
    inputs, labels = data

    outputs = best_net(inputs)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

0.9858


In [2]:
trainset = torchvision.datasets.SVHN('../SVHN/', split='train', download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize(28),
                               torchvision.transforms.Grayscale(1),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])

train_loader = torch.utils.data.DataLoader(trainset,
                             batch_size=4, shuffle=True)

val_loader = torch.utils.data.DataLoader(validationset,
                             batch_size=1, shuffle=True)


SVHN_loader = torch.utils.data.DataLoader(
  torchvision.datasets.SVHN('../SVHN/', split='test', download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize(28),
                               torchvision.transforms.Grayscale(1),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
                             batch_size=1, shuffle=True)

correct = 0
total = 0
for i, data in enumerate(SVHN_loader, 0):
    inputs, labels = data

    outputs = best_net(inputs)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

Using downloaded and verified file: ../SVHN/train_32x32.mat
Using downloaded and verified file: ../SVHN/test_32x32.mat


NameError: name 'best_net' is not defined

### Transfer learning

In [85]:
model_tf = copy.deepcopy(best_net)




for param in model_tf.parameters():
    param.requires_grad = False

#model_tf.fc2 = nn.Linear(84, 10)
for param in model_tf.fc2.parameters():
    param.requires_grad = True

best_accuracy = 0
#best_net = 0

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model_tf(inputs)
            loss = criterion(outputs, labels)
            writer.add_scalar("Loss/train", loss, epoch)
            loss.backward()
            optimizer.step()
            if(i % 100 == 99):
                print(
                    f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                    end=''
                )

    correct = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data
        labels = labels

        outputs = model_tf(inputs)
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        #best_net = copy.deepcopy(model)
        print(" (new best)")

print('Finished Training')
writer.flush()

Epoch 1 [14600/14652] - Loss: 2.4866814613342285, accuracy:  0.17915642915642915
 (new best)
Epoch 2 [14600/14652] - Loss: 2.6639719009399414, accuracy:  0.17915642915642915
Epoch 3 [10000/14652] - Loss: 2.123826503753662

KeyboardInterrupt: 

In [81]:
print(model_tf.fc2.weight)

Parameter containing:
tensor([[-0.0148, -0.1043, -0.0189,  0.0556,  0.0757,  0.0809,  0.0307, -0.0970,
         -0.1001,  0.0385, -0.0188,  0.0866, -0.0299, -0.0508, -0.0799,  0.0582,
          0.0822,  0.0041,  0.0653,  0.0345,  0.0432,  0.0683,  0.0985,  0.0739,
          0.0154,  0.0056, -0.1088, -0.0731, -0.0035,  0.0906, -0.0565,  0.0718,
         -0.0130, -0.0640,  0.0969,  0.0291, -0.0530,  0.0984,  0.0657,  0.0188,
          0.0440, -0.0284,  0.1059,  0.1013, -0.0560, -0.0397, -0.0819, -0.1056,
         -0.0844, -0.1052,  0.0151, -0.0156, -0.0195,  0.0976,  0.0970,  0.0433,
         -0.0064,  0.0876, -0.0734,  0.0170, -0.0047,  0.0397, -0.0178,  0.0128,
          0.0977, -0.0080, -0.1042, -0.0667,  0.1005,  0.0298, -0.0673, -0.0925,
          0.0671,  0.0934,  0.0719, -0.0029, -0.0010,  0.0401, -0.0389, -0.0414,
          0.0022,  0.0307, -0.0265,  0.0846],
        [ 0.1040, -0.0953, -0.0077, -0.0382,  0.0774, -0.0511, -0.1073, -0.0053,
         -0.0218, -0.0089, -0.0301, -0.04

In [82]:
print(best_net.fc2.weight)

Parameter containing:
tensor([[-3.2681e-02,  3.6461e-02,  8.8861e-02,  2.8648e-02, -6.0319e-02,
          5.3470e-02,  1.2102e-01, -1.6033e-01, -1.7290e-01,  1.0319e-01,
         -1.0360e-01,  9.5339e-02, -4.7467e-02,  9.2280e-02,  3.9894e-02,
          3.3716e-03,  1.8028e-02, -2.5105e-02,  4.6425e-02, -1.9504e-01,
         -1.6906e-01,  9.8276e-03,  1.5334e-02,  1.0193e-01, -1.3571e-01,
         -2.0996e-01,  1.0975e-01, -1.4418e-01,  8.1126e-02,  2.2326e-02,
         -2.1056e-01, -1.4813e-01, -1.1161e-01, -3.3030e-03, -3.3783e-02,
         -1.7567e-02, -1.8456e-01,  1.8220e-02,  1.1045e-01,  3.3573e-02,
         -5.4915e-02,  4.3983e-02,  7.8056e-02, -2.8809e-01, -9.7631e-02,
          6.2399e-02, -1.2876e-01, -7.8045e-02, -1.4480e-02,  6.1431e-02,
         -4.5326e-02,  4.1319e-02,  8.0601e-02,  2.4876e-01,  1.0518e-01,
         -5.2108e-02, -1.5431e-01, -1.9858e-02, -1.8752e-01, -1.2958e-01,
         -1.7929e-01,  9.6461e-02, -5.8395e-02,  7.9012e-02, -7.4403e-02,
          2.1461